---------------------------------------------
LÉKAŘI: Skript pro čištění dat
---------------------------------------------
Předem jsem si přes: assign_address_to_amd_code_PSC.py doplnila do datasetu  
kód RUIAN a název města.
Tento skript provádí:
- Exploraci dat a zjištění rozsahu a kvality dat
- Vyčištění nepotřebných sloupců
- Filtrování pouze na boxy (ne pobočky)
- Odstranění třetích stran (např. AlzaBox)
- Vyčištění názvu boxů (name), měst (city)
- Porovnání city vs. city_name z RUIAN
- Rozpad sloupce address na street a house_no
- Nakonec fixace datových typů pro merge a SQL export

V roce 2023  
5 473 ordinací praktického lékaře pro dospělé  
2 120 pro děti a dorost  
Praktičtí zubní lékaři provozovali 5 661 samostatných ordinací.

In [774]:
import pandas as pd

column_types = {
    'Obec': str,
    'Psc': str,
    'Ulice': str,
    'CisloDomovniOrientacni': str,
    'city_code': str,
    'city_name': str,
    'Lat': float,
    'Lng': float
}

data = pd.read_csv(r'C:\Users\Sabina\Sabi dokumenty\01 Czechitas\01 PYTHON\Python\Detektivky_vizualni_smog\sabi\data\Otevrena-data-NR-01-01-Mista-poskytovani (1) (2)_kod_obce.csv',
    dtype=column_types
)

df = pd.DataFrame(data)

C:\Users\Sabina\AppData\Local\Temp\ipykernel_10968\545570579.py:14: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(r'C:\Users\Sabina\Sabi dokumenty\01 Czechitas\01 PYTHON\Python\Detektivky_vizualni_smog\sabi\data\Otevrena-data-NR-01-01-Mista-poskytovani (1) (2)_kod_obce.csv',


In [775]:
df.head(2)

,MistoPoskytovaniId,ZdravotnickeZarizeniId,Kod,NazevZarizeni,DruhZarizeni,Obec,Psc,Ulice,CisloDomovniOrientacni,Kraj,...,OborPece,FormaPece,DruhPece,OdbornyZastupce,Lat,Lng,city_code,city_name,search_term,search_type
0,248893,179113,23019638000000,Mgr. Veronika Vosková,Samostatné zařízení fyzioterapeuta,Praha 10,10100,Dykova,543/31,Hlavní město Praha,...,Fyzioterapeut,ambulantní péče,NaN,NaN,50.074707,14.453731,554782.0,Praha,Dykova Praha 10 10100,full
1,248917,179111,26422557000027,"EUC Laboratoře s.r.o., odběrová místnost",Odběrová místnost,Praha 2,12800,Na slupi,448/6,Hlavní město Praha,...,"lékařská mikrobiologie, klinická biochemie, he...",ambulantní péče,NaN,Helena Machovcová,50.069243,14.420391,554782.0,Praha,Na slupi Praha 2 12800,full


In [776]:
df.shape

(40298, 37)

In [777]:
df.columns[df.nunique() == 1]

Index([], dtype='object')

In [778]:
df.columns[df.nunique() != 1]

Index(['MistoPoskytovaniId', 'ZdravotnickeZarizeniId', 'Kod', 'NazevZarizeni',
       'DruhZarizeni', 'Obec', 'Psc', 'Ulice', 'CisloDomovniOrientacni',
       'Kraj', 'KrajCode', 'Okres', 'OkresCode', 'SpravniObvod',
       'PoskytovatelTelefon', 'PoskytovatelFax', 'PoskytovatelEmail',
       'PoskytovatelWeb', 'Ico', 'TypOsoby', 'PravniFormaKod', 'KrajCodeSidlo',
       'OkresCodeSidlo', 'ObecSidlo', 'PscSidlo', 'UliceSidlo',
       'CisloDomovniOrientacniSidlo', 'OborPece', 'FormaPece', 'DruhPece',
       'OdbornyZastupce', 'Lat', 'Lng', 'city_code', 'city_name',
       'search_term', 'search_type'],
      dtype='object')

In [779]:
df = df[['NazevZarizeni','DruhZarizeni', 'FormaPece', 'OborPece', 'Obec', 'Psc', 'Ulice', 
         'CisloDomovniOrientacni', 'Kraj', 'Lat', 'Lng', 'city_code', 'city_name']]
df.head(3)

,NazevZarizeni,DruhZarizeni,FormaPece,OborPece,Obec,Psc,Ulice,CisloDomovniOrientacni,Kraj,Lat,Lng,city_code,city_name
0,Mgr. Veronika Vosková,Samostatné zařízení fyzioterapeuta,ambulantní péče,Fyzioterapeut,Praha 10,10100,Dykova,543/31,Hlavní město Praha,50.074707,14.453731,554782.0,Praha
1,"EUC Laboratoře s.r.o., odběrová místnost",Odběrová místnost,ambulantní péče,"lékařská mikrobiologie, klinická biochemie, he...",Praha 2,12800,Na slupi,448/6,Hlavní město Praha,50.069243,14.420391,554782.0,Praha
2,MDDr. Natalie Malinová,Samostatná ordinace PL - stomatologa,ambulantní péče,zubní lékařství,Praha 10,10100,Smolenská,42/29,Hlavní město Praha,50.069284,14.451211,554782.0,Praha


In [780]:
df['DruhZarizeni'].dropna().unique().tolist()

['Samostatné zařízení fyzioterapeuta',
 'Odběrová místnost',
 'Samostatná ordinace PL - stomatologa',
 'Samostatné zařízení nelékaře - jiné',
 'Samost. ordinace všeob. prakt. lékaře',
 'Lékárna',
 'Samostatná ordinace lékaře specialisty',
 'Centrum duševního zdraví',
 'Samostatná stomatologická laboratoř',
 'Sam.ord.prakt.lékaře pro děti a dorost',
 'Oční optika',
 'Samostatná odborná laboratoř',
 'Hospic',
 'Domácí zdravotní péče',
 'Výjezdová skupina záchranné služby',
 'Poskytovatel amb. služeb (nad 5 oborů), Samostatná ordinace PL - gynekologa',
 'Samostatná ordinace PL - gynekologa',
 'Výdejna zdravotnických prostředků',
 'Poskytovatel amb. služeb (do 5 oborů)',
 'Nemocnice následné péče',
 'Poskytovatel amb. služeb (nad 5 oborů)',
 'Poskytovatel amb. služeb (do 5 oborů), Sam.ord.prakt.lékaře pro děti a dorost, Samostatné zařízení psychologa',
 'Poskytovatel amb. služeb (do 5 oborů), Samostatná ordinace PL - gynekologa',
 'Poskytovatel amb. služeb (nad 5 oborů), Samostatná ordinac

In [781]:
df['FormaPece'].dropna().unique().tolist()

['ambulantní péče',
 'zdrav. péče poskytovaná ve vlastním soc. prostředí pacienta - domácí péče - ošetřovatelská',
 'jednodenní péče',
 'ambulantní péče, specializovaná ambulantní péče, zdrav. péče poskytovaná ve vlastním soc. prostředí pacienta - domácí péče - ošetřovatelská, zdrav. péče poskytovaná ve vlastním soc. prostředí pacienta - návštěvní služba',
 'ambulantní péče, zdrav. péče poskytovaná ve vlastním soc. prostředí pacienta - domácí péče - ošetřovatelská',
 'specializovaná ambulantní péče',
 'ambulantní péče, jednodenní péče',
 'zdrav. péče poskytovaná ve vlastním soc. prostředí pacienta - domácí péče - léčebně rehabilitační',
 'dlouhodobá lůžková paliativní péče',
 'zdrav. péče poskytovaná ve vlastním soc. prostředí pacienta - domácí péče - ošetřovatelská, zdrav. péče poskytovaná ve vlastním soc. prostředí pacienta - domácí péče - paliativní',
 'primární ambulantní péče',
 'ambulantní péče, zdrav. péče poskytovaná ve vlastním soc. prostředí pacienta - návštěvní služba',
 'dl

In [782]:
df['OborPece'].dropna().unique().tolist()

['Fyzioterapeut',
 'lékařská mikrobiologie, klinická biochemie, hematologie a transfúzní lékařství, alergologie a klinická imunologie',
 'zubní lékařství',
 'Dentální hygienistka',
 'Porodní asistentka',
 'všeobecné praktické lékařství',
 'praktické lékárenství',
 'anesteziologie a intenzivní medicína',
 'Klinický psycholog, Klinický psycholog, Sestra pro péči v psychiatrii, Sestra pro péči v psychiatrii, dětská a dorostová psychiatrie, dětská a dorostová psychiatrie, psychiatrie, psychiatrie',
 'Fyzioterapeut, Fyzioterapeut',
 'všeobecné praktické lékařství, vnitřní lékařství',
 'oftalmologie',
 'Zubní technik',
 'rehabilitační a fyzikální medicína',
 'chirurgie',
 'praktické lékařství pro děti a dorost',
 'Dentální hygienistka, zubní lékařství',
 'anesteziologie a intenzivní medicína, anesteziologie a intenzivní medicína',
 'vnitřní lékařství, revmatologie',
 'Optometrista',
 'klinická biochemie',
 'paliativní medicína',
 'Všeobecná sestra',
 'dětská chirurgie',
 'Všeobecná sestra, u

In [783]:
# 1. Pomlčky
df['DruhZarizeni_clean'] = df['DruhZarizeni'].astype(str).str.replace('–', '-', regex=False)

# 2. Typy ordinací
praktici = 'Samost. ordinace všeob. prakt. lékaře'
pediatri = 'Sam.ord.prakt.lékaře pro děti a dorost'
zubari = 'Samostatná ordinace PL - stomatologa'

# 3. Filtrování typů ordinací
f1 = df[
    df['DruhZarizeni_clean'].str.contains(praktici, na=False) |
    df['DruhZarizeni_clean'].str.contains(pediatri, na=False) |
    df['DruhZarizeni_clean'].str.contains(zubari, na=False)
]

# 4. Filtrování formy péče
f2 = f1[f1['FormaPece'].str.contains('ambulantní', na=False)]

# 5. Filtrování zakázaných slov (správně)
zakazane = ['nemocnice', 'klinika', 'hospic', 'ústav', 'centrum', 'sanatorium',
            'lékárna', 'stacionář', 'léčebna', 'plazmy', 'specialist',
            'zařízení', 'středisko', 'ostatní', 'poskytovatel', 'gynekolog']

f3 = f2[~f2['DruhZarizeni'].fillna('').str.lower().str.contains('|'.join(zakazane))]

# 6. Filtr podle oborů
povolene_obory = ['všeobecné praktické lékařství', 'pediatrie',
                  'praktické lékařství pro děti a dorost', 'zubní lékařství']
f4 = f3[f3['OborPece'].isna() | f3['OborPece'].str.contains('|'.join(povolene_obory), na=False)]

# Výsledné data
filtered = f4.reset_index(drop=True)
filtered.head(100)

,NazevZarizeni,DruhZarizeni,FormaPece,OborPece,Obec,Psc,Ulice,CisloDomovniOrientacni,Kraj,Lat,Lng,city_code,city_name,DruhZarizeni_clean
0,MDDr. Natalie Malinová,Samostatná ordinace PL - stomatologa,ambulantní péče,zubní lékařství,Praha 10,10100,Smolenská,42/29,Hlavní město Praha,50.069284,14.451211,554782.0,Praha,Samostatná ordinace PL - stomatologa
1,MUDr. Stanislav Masný,Samost. ordinace všeob. prakt. lékaře,ambulantní péče,všeobecné praktické lékařství,Sušice,34201,Chmelenská,117,Plzeňský kraj,49.230033,13.526477,557153.0,Sušice,Samost. ordinace všeob. prakt. lékaře
2,MEDILIFE s.r.o.,Samost. ordinace všeob. prakt. lékaře,ambulantní péče,všeobecné praktické lékařství,Česká Lípa,47001,Mariánská,607/18,Liberecký kraj,50.687602,14.538560,561380.0,Česká Lípa,Samost. ordinace všeob. prakt. lékaře
3,Závodní lékaři s. r. o.,Samost. ordinace všeob. prakt. lékaře,ambulantní péče,"všeobecné praktické lékařství, vnitřní lékařství",Ústí nad Labem,40007,U Pivovarské zahrady,793/5,Ústecký kraj,50.667658,14.075887,554804.0,Ústí nad Labem,Samost. ordinace všeob. prakt. lékaře
4,MUDr. Alaa M A Sara SHAMASNA,Samost. ordinace všeob. prakt. lékaře,ambulantní péče,všeobecné praktické lékařství,Most,43401,Moskevská,508,Ústecký kraj,50.507738,13.656654,567027.0,Most,Samost. ordinace všeob. prakt. lékaře
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Praktik HB s.r.o.,Samost. ordinace všeob. prakt. lékaře,ambulantní péče,všeobecné praktické lékařství,Šlapanov,58251,NaN,40,Kraj Vysočina,49.542907,15.657297,NaN,NaN,Samost. ordinace všeob. prakt. lékaře
96,Praktik HB s.r.o.,Samost. ordinace všeob. prakt. lékaře,ambulantní péče,všeobecné praktické lékařství,Štoky,58253,NaN,458,Kraj Vysočina,49.504168,15.587395,NaN,NaN,Samost. ordinace všeob. prakt. lékaře
97,Yoursmile s.r.o.,Samostatná ordinace PL - stomatologa,ambulantní péče,zubní lékařství,Ostrava,70200,Zelená,2502/37,Moravskoslezský kraj,49.828544,18.272216,554821.0,Ostrava,Samostatná ordinace PL - stomatologa
98,Domov pro seniory Zahradní Město - ordinace,Samostatná ordinace PL - stomatologa,ambulantní péče,zubní lékařství,Praha 10,10600,Sněženková,2973/8,Hlavní město Praha,50.054497,14.501786,554782.0,Praha,Samostatná ordinace PL - stomatologa


In [784]:
# 1. Připravíme si nové sloupce s kategoriemi podle 'DruhZarizeni_clean'
filtered['profese'] = filtered['DruhZarizeni_clean'].apply(
    lambda x: 'praktik' if 'prakt. lékaře' in x and 'děti' not in x else
              'pediatr' if 'děti a dorost' in x else
              'zubař' if 'stomatologa' in x else 'jiné'
)

# 2. Spočítáme počet řádků (pracovišť) podle profese
vysledky = filtered['profese'].value_counts()

# 3. Pro jistotu výstup jako DataFrame
vysledky_df = vysledky.reset_index()
vysledky_df.columns = ['profese', 'pocet_radku']
print(vysledky_df)

   profese  pocet_radku
0    zubař         5526
1  praktik         5344
2  pediatr         2016


In [785]:
# Slovník pro převod textu na ID
mapa_profese = {
    'praktik': 1,
    'pediatr': 2,
    'zubař': 3
}

# Vytvoření sloupců na základě již existujícího sloupce 'profese'
filtered['id_typ_profese'] = filtered['profese'].map(mapa_profese)
filtered['nazev_profese'] = filtered['profese']

# Kontrola výsledku
filtered[['DruhZarizeni_clean', 'id_typ_profese', 'nazev_profese']].head()

,DruhZarizeni_clean,id_typ_profese,nazev_profese
0,Samostatná ordinace PL - stomatologa,3,zubař
1,Samost. ordinace všeob. prakt. lékaře,1,praktik
2,Samost. ordinace všeob. prakt. lékaře,1,praktik
3,Samost. ordinace všeob. prakt. lékaře,1,praktik
4,Samost. ordinace všeob. prakt. lékaře,1,praktik


In [786]:
# filtered['city_code'] = filtered['city_code'].astype(str).str.replace(r'\.0$', '', regex=True)
# filtered['city_code'].head()
chybi_city_code = df[df['city_code'].isna()]
pocet = len(chybi_city_code)
print(pocet)

2688


In [787]:
chybi_city_code = df[df['city_code'].isna()]
pocet = len(chybi_city_code)
print(pocet)


2688


In [788]:
chybi_city_code

,NazevZarizeni,DruhZarizeni,FormaPece,OborPece,Obec,Psc,Ulice,CisloDomovniOrientacni,Kraj,Lat,Lng,city_code,city_name,DruhZarizeni_clean
3,"Kristýna Šmahelová, DiS.",Samostatné zařízení nelékaře - jiné,ambulantní péče,Dentální hygienistka,Vojnův Městec,59101,NaN,81,Kraj Vysočina,49.680466,15.878536,NaN,NaN,Samostatné zařízení nelékaře - jiné
30,Pediatrie Vydrová s.r.o.,Sam.ord.prakt.lékaře pro děti a dorost,ambulantní péče,praktické lékařství pro děti a dorost,Holoubkov,33801,NaN,7,Plzeňský kraj,49.774913,13.692348,NaN,NaN,Sam.ord.prakt.lékaře pro děti a dorost
53,LZS ZZSHMP Praha 6 - Ruzyně - Letiště Václava ...,Výjezdová skupina záchranné služby,ambulantní péče,"Všeobecná sestra, urgentní medicína",Praha 6,NaN,K letišti,NaN,Hlavní město Praha,NaN,NaN,NaN,NaN,Výjezdová skupina záchranné služby
55,Výjezdová základna ZZSHMP Praha 5,Výjezdová skupina záchranné služby,NaN,"Všeobecná sestra, urgentní medicína",Praha,NaN,Vackova,NaN,Hlavní město Praha,NaN,NaN,NaN,NaN,Výjezdová skupina záchranné služby
71,"JANA TROLLEROVÁ, DiS.",Samostatné zařízení nelékaře - jiné,ambulantní péče,Dentální hygienistka,Vacov,38486,NaN,65,Jihočeský kraj,49.136100,13.727758,NaN,NaN,Samostatné zařízení nelékaře - jiné
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40242,"Sdružení ozdravoven a léčeben okresu Trutnov, ...",Dětská ozdravovna,NaN,dětské lékařství,Pec pod Sněžkou,54221,NaN,185,Královéhradecký kraj,50.689780,15.730352,NaN,NaN,Dětská ozdravovna
40250,Fakultní nemocnice Hradec Králové,"Fakultní nemocnice, Porodnice, Samost. ordinac...","akutní lůžková péče intenzivní, akutní lůžková...","gynekologie dětí a dospívajících, gynekologie ...",Hradec Králové,50005,Sokolská,581,Královéhradecký kraj,NaN,NaN,NaN,NaN,"Fakultní nemocnice, Porodnice, Samost. ordinac..."
40253,"Parkhotel Golf Mariánské Lázně, a. s.",Samostatná ordinace lékaře specialisty,ambulantní péče,"Fyzioterapeut, rehabilitační a fyzikální medicína",Mariánské Lázně,35301,NaN,580,Karlovarský kraj,49.975423,12.735965,NaN,NaN,Samostatná ordinace lékaře specialisty
40266,Rehabilitační ústav Kladruby,Rehabilitační ústav,"ambulantní péče, následná lůžková léčebně reha...","Fyzioterapeut, Fyzioterapeut, Klinický logoped...",Kladruby,25801,NaN,30,Středočeský kraj,49.721828,14.958901,NaN,NaN,Rehabilitační ústav


tady dělám mezikrok, ukládám soubor do pomocného csv  
z toho csv vytahuji jen chybějící záznamy city_code  
[text](data/doctors_pomocny_only_lines_missing_city_code_empty.csv)
zkouším spustit (assign_address_to_amd_code_PSC.py) - nepřináší další záznamy  
-- zkouším v různých variantách   
zkouším skript (merge_city_code_where_missing_offline.py) - tahá data ze souboru  
který zpracovala iva p (combined_addresses_cz_cleaned), tedy offline párování  
výsledkem je 100% shoda u těch chybějících, namátkově zkontroluji správnost  
a dataset propustím (je lepší mít malou chybovost než dropovat 1500 řádků)  
následující kód ponechám, chci se po dokončení projektu k celému procesu vrátit  
a nechci ztratit posloupnost kroků
[text](data/doctors_pomocny_only_lines_missing_city_codes_filled.csv)
nyní potřebuji z výše uvedeného souboru načíst COL 'city_codes_merged'. 
Předcházející krok jsem záměrně udělala jen pro tu část datasetu, kde chyběly  
údaje. Vím, že online vyhledávání bylo preciznější a vykazovalo méně chyb.  
Domnívám se, že tak dosáhnu lepšího výsledku. Stále je ve hře možnost, že vyhodím   
1.400 řádků, kde chybí city_code (1.400 řádků pro mé vybrané 3 kategorie, 
bez nemocnic atd)


In [789]:
import csv

# Sloupce, kde chceme chránit obsah (např. s adresami a městy)
sloupce_textove = ['Ulice', 'Obec', 'Psc', 'DruhZarizeni', 'FormaPece', 'OborPece', 'NazevZarizeni']

for sloupec in sloupce_textove:
    if sloupec in filtered.columns:
        filtered[sloupec] = filtered[sloupec].astype(str)

# Uložit CSV s uvozovkami
filtered.to_csv(
    r'C:\Users\Sabina\Sabi dokumenty\01 Czechitas\01 PYTHON\Python\Detektivky_vizualni_smog\sabi\data\doctors_pomocny.csv',
    index=False,
    encoding='utf-8-sig',   # BOM pro lepší kompatibilitu s Excel / Sheets
    quoting=csv.QUOTE_ALL,  # obalit všechny texty do uvozovek
    quotechar='"',
    sep=','
)

dohledáváním chybějících kódů jsem ztratila příliš mnoho času. nedaří se mi 
nachystat správný kód, který by mi z tabulky 
"doctors_pomocny_only_lines_missing_city_codes_filled.csv" doplnil do souboru  
docotrs_pomocne.csv zbvývající kódy (z offline vyhledávání) .... nemám čas...  
a tak se uchyluji k ručnímu doplnění
snad to stihnu opravit po dokončení projektu a nikdo si této prasárny nevšimne  

In [790]:
filtered['id'] = range(1, len(filtered) + 1)
filtered['id'] = filtered['id'].astype(int)

In [791]:
# Slovník: id → city_code
doplnit_city_code = {
1223:	"505781",
2206:	"505781",
3997:	"505781",
2443:	"538108",
4407:	"539040",
103:	"539660",
5652:	"539660",
7599:	"571211",
7453:	"539368",
10792:	"539368",
5454:	"530689",
108:	"531031",
2860:	"531049",
4173:	"529796",
9034:	"529796",
9035:	"529796",
12331:	"530352",
5895:	"529958",
12337:	"529958",
7034:	"530948",
12038:	"530948",
12048:	"530948",
11143:	"530115",
2032:	"529991",
5433:	"530310",
5602:	"530310",
1387:	"529648",
1792:	"529648",
1416:	"530158",
7035:	"530166",
11984:	"530166",
7033:	"530701",
7025:	"530344",
11132:	"598381",
5779:	"540714",
895:	"541206",
6984:	"540811",
6982:	"540285",
6983:	"539970",
3100:	"541231",
7776:	"541231",
937:	"540315",
9635:	"540315",
1689:	"541044",
9225:	"541044",
9605:	"541044",
429:	"540552",
2398:	"540552",
11134:	"541516",
11133:	"540129",
3175:	"540439",
7787:	"540439",
9803:	"541311",
11131:	"541320",
11135:	"541320",
9802:	"540536",
10538:	"541613",
4172:	"540498",
9224:	"540846",
12683:	"533203",
11833:	"531545",
11835:	"531545",
9880:	"531316",
11834:	"531316",
12681:	"531782",
677:	"531472",
731:	"531472",
1619:	"531472",
7983:	"531472",
8962:	"531154",
12675:	"532002",
7728:	"541893",
9156:	"541893",
4041:	"542458",
4042:	"541907",
4040:	"541770",
4636:	"544248",
1287:	"541982",
1288:	"541982",
3425:	"598526",
2026:	"542032",
1690:	"542415",
12071:	"542415",
12078:	"542105",
2839:	"532720",
7173:	"532720",
3250:	"533009",
1559:	"532681",
4762:	"532142",
4163:	"533068",
9248:	"533068",
1908:	"534765",
5909:	"534765",
5907:	"534790",
9569:	"534790",
5908:	"534731",
8951:	"535303",
9070:	"535338",
758:	"571792",
8313:	"533572",
10346:	"533629",
9158:	"533181",
10022:	"533181",
12360:	"533181",
706:	"534099",
6101:	"533955",
7668:	"533955",
5456:	"534439",
683:	"528196",
1848:	"534129",
1847:	"534030",
705:	"534226",
384:	"534617",
2970:	"534617",
11263:	"534617",
5846:	"534021",
6814:	"534021",
7800:	"534021",
6813:	"531278",
10902:	"534161",
1182:	"534331",
5845:	"534331",
6622:	"534331",
10903:	"534331",
2408:	"534625",
995:	"534102",
5633:	"530999",
6812:	"534315",
2082:	"537080",
7059:	"537080",
8567:	"537578",
4175:	"538035",
2218:	"537179",
414:	"537357",
5438:	"537357",
1209:	"537721",
9519:	"537721",
1210:	"537781",
9351:	"537781",
7342:	"537411",
10485:	"537411",
833:	"535583",
11241:	"535583",
23:	"536971",
10401:	"535630",
11249:	"536636",
12234:	"536636",
4793:	"566039",
11250:	"566039",
380:	"536610",
7815:	"535451",
10404:	"535451",
5834:	"536661",
5198:	"536130",
4206:	"558915",
4210:	"558915",
2066:	"559504",
8399:	"559628",
10080:	"559628",
8571:	"559059",
8572:	"559059",
4569:	"559334",
9545:	"559334",
4389:	"559091",
4568:	"559563",
6556:	"558656",
3399:	"559041",
4208:	"559237",
11718:	"559237",
11725:	"566969",
10829:	"558796",
11724:	"558796",
12832:	"558796",
1073:	"559202",
1074:	"559202",
3139:	"559202",
3713:	"559261",
10828:	"559261",
8493:	"559695",
11709:	"559695",
11719:	"559695",
5497:	"558460",
7223:	"558460",
2320:	"558001",
5415:	"540340",
6800:	"558117",
6801:	"558141",
7224:	"558141",
3716:	"557846",
4212:	"558401",
11077:	"557731",
11076:	"540668",
549:	"557668",
742:	"557668",
5473:	"558061",
5564:	"557862",
7323:	"557862",
10387:	"557862",
7097:	"558630",
7:	"559814",
2363:	"559814",
2430:	"559814",
1167:	"559857",
1701:	"559903",
3634:	"560006",
6152:	"541061",
11777:	"541061",
9881:	"560057",
1700:	"559725",
6653:	"559725",
6654:	"559725",
6655:	"559733",
8308:	"559750",
8475:	"559776",
1189:	"560162",
10469:	"556106",
7419:	"556831",
3375:	"556751",
4195:	"556912",
10472:	"556912",
1905:	"556467",
2775:	"556467",
11434:	"557366",
11437:	"557366",
4209:	"557111",
3038:	"556181",
3373:	"556181",
8915:	"557129",
10222:	"557544",
11727:	"557544",
7073:	"554120",
7065:	"553794",
9718:	"553794",
12839:	"553794",
12837:	"553450",
39:	"560758",
8839:	"560758",
614:	"503916",
5176:	"554677",
5993:	"554880",
3015:	"554502",
11733:	"554502",
11750:	"554502",
3926:	"560324",
12617:	"560502",
11081:	"560413",
10814:	"556947",
4202:	"555363",
4705:	"555347",
10304:	"555347",
10819:	"555215",
10821:	"555622",
5610:	"544795",
8631:	"544795",
4095:	"535664",
2632:	"545155",
9808:	"545155",
11522:	"535877",
7995:	"544884",
1838:	"544515",
11511:	"544329",
11515:	"545368",
9518:	"535991",
11517:	"535991",
11516:	"545015",
9108:	"544892",
4214:	"544426",
9109:	"544426",
12023:	"544426",
5823:	"544442",
11502:	"544442",
11512:	"544442",
7996:	"545082",
11521:	"545376",
9563:	"545881",
12263:	"545881",
12258:	"546500",
12242:	"547239",
919:	"547204",
12230:	"546143",
9761:	"546054",
12246:	"547212",
12248:	"547212",
6158:	"546097",
5395:	"545392",
9547:	"513661",
9548:	"545431",
3231:	"545546",
9549:	"545546",
1250:	"545813",
1341:	"545767",
6763:	"545520",
24:	"545457",
12581:	"545449",
10318:	"545465",
10582:	"545465",
12586:	"545465",
9967:	"545848",
1343:	"545601",
5063:	"545601",
10743:	"545601",
25:	"545597",
6359:	"545597",
805:	"545481",
1368:	"545481",
10744:	"545481",
1837:	"545406",
12582:	"545406",
12579:	"545694",
1835:	"545619",
12583:	"545619",
12580:	"545775",
10581:	"545503",
12585:	"545503",
3734:	"550329",
10741:	"550540",
2404:	"550353",
8402:	"550728",
11998:	"550728",
3290:	"550515",
5052:	"550167",
6922:	"550167",
7244:	"550167",
7243:	"550621",
2446:	"551163",
2447:	"551562",
10316:	"551571",
6161:	"551961",
12004:	"550957",
7468:	"550906",
5267:	"550850",
10313:	"536466",
10314:	"536466",
3230:	"552470",
3229:	"553239",
2835:	"552208",
1369:	"552101",
6157:	"552992",
11985:	"552992",
757:	"552801",
5888:	"552801",
9186:	"552801",
12288:	"552801",
11986:	"553140",
1764:	"552666",
2865:	"552666",
5327:	"552666",
5318:	"548464",
5317:	"549142",
12013:	"548472",
12029:	"548472",
2646:	"548456",
11115:	"548456",
5758:	"548171",
2762:	"549215",
8523:	"547778",
12014:	"547778",
8111:	"548774",
2645:	"547581",
11114:	"547581",
5282:	"548391",
3108:	"549380",
5499:	"549339",
8221:	"549339",
2348:	"549258",
9417:	"550027",
9416:	"549495",
9288:	"549347",
756:	"549843",
7351:	"549517",
11497:	"549517",
6027:	"549452",
3308:	"568023",
4171:	"568201",
8907:	"568201",
5243:	"555223",
1539:	"568058",
5245:	"568058",
9637:	"568309",
5244:	"568147",
8153:	"562335",
10230:	"555193",
6435:	"546496",
12820:	"546348",
12821:	"562556",
2490:	"562530",
1433:	"562793",
5923:	"562793",
10232:	"562432",
10289:	"562432",
10017:	"562912",
3580:	"562751",
3637:	"562751",
11979:	"562751",
1435:	"562947",
9376:	"562947",
9375:	"562661",
5621:	"565237",
647:	"565709",
12197:	"565415",
12184:	"565393",
12179:	"565105",
12180:	"565211",
12181:	"565121",
6684:	"565431",
2527:	"565679",
4454:	"565679",
2902:	"563471",
157:	"563358",
1252:	"563358",
5919:	"563323",
9622:	"563102",
10431:	"567141",
3130:	"567043",
10432:	"567043",
3547:	"567329",
5335:	"567329",
11969:	"567329",
747:	"567256",
9360:	"566535",
10019:	"566535",
12200:	"566284",
67:	"566454",
2501:	"566357",
9630:	"566217",
10523:	"566217",
10522:	"566667",
6425:	"566152",
6233:	"566501",
8360:	"566306",
4964:	"564176",
709:	"564354",
768:	"564354",
2165:	"564354",
4655:	"564354",
4960:	"564184",
4961:	"564184",
4965:	"564184",
9750:	"564397",
10428:	"564397",
11474:	"564541",
8251:	"564044",
11475:	"564044",
5460:	"564494",
2252:	"563781",
8711:	"563595",
11083:	"563595",
7542:	"563749",
7807:	"563749",
333:	"563706",
2063:	"563706",
4956:	"563706",
5276:	"563871",
10692:	"563871",
10693:	"563871",
2520:	"563561",
9508:	"563561",
2521:	"563854",
10687:	"563854",
2335:	"546585",
3561:	"546585",
11084:	"563633",
8808:	"563838",
12775:	"563757",
1289:	"563668",
2768:	"563692",
9894:	"563692",
3674:	"562246",
3672:	"561410",
5590:	"562025",
9948:	"561444",
4730:	"570044",
9047:	"570044",
365:	"571091",
9532:	"571091",
10225:	"571091",
297:	"569976",
295:	"570958",
296:	"570435",
8362:	"570249",
6314:	"570206",
7593:	"569968",
11004:	"569968",
392:	"570231",
11003:	"570231",
2100:	"569917",
8715:	"569917",
9309:	"570265",
6104:	"572659",
11881:	"573469",
11880:	"573639",
7384:	"573124",
2572:	"573001",
1408:	"573094",
10735:	"573248",
11870:	"573248",
9367:	"577219",
9850:	"577294",
11038:	"577294",
1915:	"577171",
9168:	"577006",
1505:	"577499",
6474:	"577499",
335:	"577553",
1506:	"577553",
12493:	"577553",
334:	"574201",
12492:	"574201",
2089:	"577120",
3820:	"577405",
7869:	"577405",
3821:	"577162",
6038:	"577162",
2180:	"577456",
3491:	"577456",
8993:	"577456",
4019:	"577456",
1290:	"577081",
12500:	"577081",
9258:	"577596",
3144:	"577413",
4959:	"577413",
6480:	"577421",
6473:	"577120",
12496:	"577553",
2188:	"576425",
7604:	"576425",
5645:	"576778",
12505:	"576468",
12504:	"576581",
10549:	"576786",
10553:	"576786",
12329:	"576786",
8949:	"576573",
787:	"576247",
1721:	"576247",
12325:	"576263",
3513:	"575437",
6029:	"555134",
8149:	"555134",
9552:	"555134",
9830:	"555134",
9995:	"555134",
10259:	"555134",
6708:	"575577",
6710:	"574848",
10720:	"574848",
11535:	"574848",
2725:	"574864",
3181:	"575551",
11531:	"575551",
6360:	"575534",
6705:	"575704",
4592:	"555134",
9987:	"555134",
9988:	"575593",
3384:	"574911",
10721:	"574911",
4099:	"574783",
6707:	"575089",
9993:	"571202",
5809:	"572179",
5810:	"572179",
5883:	"572578",
9469:	"572578",
12705:	"572411",
4101:	"571181",
3474:	"571776",
6271:	"571661",
9208:	"571661",
10378:	"571920",
4447:	"571831",
10738:	"571831",
12798:	"571831",
239:	"579050",
11889:	"579050",
3592:	"579335",
3593:	"579629",
4469:	"579629",
3591:	"579378",
10811:	"579581",
837:	"579548",
5910:	"579548",
8126:	"579785",
2316:	"579432",
8823:	"579432",
1555:	"579742",
2857:	"579742",
10727:	"579696",
94:	"579645",
3848:	"579645",
6114:	"579645",
2731:	"579157",
9849:	"579157",
10264:	"579327",
9452:	"579181",
9453:	"579190",
630:	"579815",
2081:	"579211",
3959:	"579211",
8445:	"579211",
1214:	"579068",
3947:	"579068",
4648:	"579068",
11900:	"579068",
4647:	"579556",
11899:	"579556",
789:	"573892",
9415:	"574082",
6313:	"574465",
9758:	"547786",
3590:	"574538",
4665:	"574210",
12816:	"574589",
7543:	"574112",
9840:	"574112",
7541:	"574074",
6014:	"580031",
28:	"580121",
7269:	"580121",
9554:	"580121",
10533:	"580074",
7164:	"580848",
7165:	"581046",
7163:	"579980",
572:	"580295",
3029:	"581178",
6020:	"581178",
9747:	"581178",
2932:	"580112",
7257:	"580112",
9812:	"580112",
3030:	"579971",
6021:	"579971",
9771:	"579971",
9811:	"580279",
11032:	"580279",
89:	"580015",
10535:	"580015",
12637:	"580015",
9202:	"580651",
10532:	"573426",
1425:	"580325",
6270:	"580325",
9983:	"580325",
6958:	"572691",
2579:	"577863",
10163:	"577863",
11863:	"577863",
6957:	"578657",
4839:	"578231",
9822:	"578487",
3447:	"577871",
4134:	"577871",
10074:	"578282",
4833:	"578789",
4862:	"578789",
1876:	"578380",
2156:	"578380",
11026:	"578380",
4097:	"578428",
11028:	"578428",
10072:	"577936",
4445:	"577944",
4444:	"578894",
8990:	"578894",
10451:	"578894",
4111:	"577995",
4130:	"577995",
5813:	"577995",
9228:	"578738",
10453:	"578738",
9229:	"578355",
10452:	"578355",
10297:	"578673",
10299:	"578673",
1815:	"577839",
9616:	"577839",
10296:	"578185",
1814:	"578878",
11025:	"578975",
8421:	"568708",
2765:	"569186",
9116:	"569186",
5548:	"569046",
6728:	"568601",
4881:	"569062",
12314:	"569062",
2766:	"569615",
96:	"569585",
97:	"569593",
229:	"569593",
1734:	"568678",
5944:	"568678",
6577:	"569691",
4012:	"569038",
8164:	"568503",
9180:	"568503",
1176:	"569089",
10041:	"569089",
10201:	"569089",
12313:	"569712",
1476:	"586846",
4224:	"586846",
7098:	"586846",
10310:	"587095",
10311:	"587621",
349:	"587346",
10608:	"587346",
713:	"587249",
5067:	"587931",
9846:	"587931",
7651:	"588121",
10312:	"588121",
5855:	"587796",
7932:	"587796",
2763:	"587401",
8413:	"587923",
10380:	"587923",
4910:	"596566",
5519:	"597091",
2665:	"595594",
5592:	"595918",
12749:	"596787",
12755:	"596787",
9665:	"596981",
2088:	"596205",
9305:	"596205",
9664:	"596205",
2041:	"595292",
12739:	"595292",
3024:	"596680",
1680:	"595454",
2522:	"595535",
5546:	"595535",
5513:	"596655",
3022:	"596825",
11157:	"596825",
11163:	"596825",
12762:	"596825",
11156:	"596141",
11162:	"596141",
3157:	"595268",
6828:	"595268",
2718:	"595551",
6247:	"596175",
6274:	"596175",
2524:	"596884",
5756:	"596884",
2523:	"596612",
6428:	"596612",
11161:	"597074",
2779:	"596574",
12759:	"596574",
2778:	"596361",
12757:	"596361",
3514:	"596345",
6136:	"596345",
374:	"595527",
2263:	"595527",
10098:	"595527",
11158:	"595527",
2262:	"597171",
2777:	"595641",
12758:	"595641",
6283:	"582794",
6290:	"583197",
7892:	"583197",
6289:	"583537",
10956:	"583634",
10959:	"583634",
2362:	"583405",
6583:	"583405",
10943:	"583405",
5860:	"584037",
3792:	"582913",
7981:	"582913",
11375:	"582972",
1263:	"583910",
1863:	"583910",
3676:	"583413",
541:	"583995",
7438:	"583995",
10147:	"583995",
7436:	"583448",
9110:	"583448",
1935:	"583383",
4524:	"583383",
218:	"582948",
4421:	"582948",
11373:	"582948",
6337:	"584185",
2644:	"583464",
7135:	"594172",
9234:	"594172",
7676:	"594911",
8599:	"593753",
7337:	"594881",
7336:	"595128",
27:	"594920",
1756:	"594920",
6379:	"593974",
6377:	"594148",
7334:	"594822",
12474:	"594431",
12487:	"594431",
8025:	"594091",
12473:	"594091",
8026:	"595071",
12480:	"595071",
5231:	"594938",
12463:	"594938",
6306:	"594202",
6502:	"594202",
7555:	"594202",
7885:	"594202",
12475:	"594202",
12476:	"594202",
12483:	"594202",
12486:	"594776",
1178:	"594121",
1179:	"593770",
1177:	"594601",
4127:	"594709",
6294:	"594709",
12485:	"594709",
7265:	"594334",
7287:	"593826",
9643:	"593826",
6378:	"594032",
7134:	"594032",
4361:	"594113",
6702:	"594768",
3102:	"594377",
4517:	"594563",
7581:	"594563",
4118:	"594229",
4158:	"594229",
4159:	"594962",
5391:	"591939",
3881:	"590401",
4112:	"590401",
11908:	"590401",
3880:	"591637",
4113:	"591637",
5392:	"590860",
5051:	"590436",
11902:	"591611",
5306:	"591432",
9353:	"591432",
5307:	"591319",
4116:	"590843",
7159:	"590843",
9582:	"590843",
11910:	"591254",
9307:	"590410",
10257:	"590568",
7472:	"590746",
9308:	"591009",
2469:	"590576",
3617:	"590576",
2259:	"590517",
4225:	"590576",
11332:	"590576",
2612:	"591629",
5366:	"591190",
8798:	"591190",
9922:	"591190",
5372:	"590991",
3260:	"591173",
11327:	"591173",
11903:	"591831",
1351:	"581828",
4578:	"581828",
11673:	"581828",
1360:	"582352",
10920:	"582352",
6592:	"582182",
12735:	"582182",
2431:	"581950",
3212:	"581950",
12510:	"581950",
4157:	"581968",
9191:	"581739",
4801:	"582468",
9891:	"582468",
12511:	"582468",
3782:	"581330",
4798:	"581330",
9721:	"582018",
7375:	"593001",
11924:	"593001",
11925:	"593001",
11930:	"593192",
11931:	"593567",
2087:	"593605",
7666:	"593508",
1350:	"592927",
7609:	"593419",
9777:	"593419",
9592:	"593087",
9593:	"550108",
11154:	"550108",
2393:	"550825",
10244:	"550825",
12392:	"592293",
7093:	"592501",
3543:	"592013",
1670:	"592234",
11358:	"592706",
11359:	"592706",
9581:	"592226",
6321:	"592391",
7309:	"592391",
702:	"592722",
198:	"592064",
9857:	"592064",
11363:	"592064",
1669:	"592862",
6222:	"592862",
7896:	"592692",
682:	"592030",
3773:	"592030",
7895:	"592030",
7897:	"592269",
7898:	"592269",
9161:	"592382",
9163:	"592382",
9872:	"592404",
9320:	"592081",
3624:	"592455",
5289:	"592455",
10665:	"592455",
7799:	"592676",
10483:	"592676",
1882:	"592200",
5005:	"592803",
9667:	"592803",
9704:	"592803",
6991:	"592510",
7432:	"592510",
7445:	"592510",
7793:	"592510",
8003:	"592510",
12394:	"592510",
7444:	"592412",
12114:	"592471",
4916:	"592285",
7367:	"592285",
6854:	"592641",
3061:	"592021",
10084:	"592021",
10195:	"592021",
6443:	"592111",
9871:	"592480",
1883:	"592820",
3790:	"592820",
10086:	"592820",
11366:	"592820",
4560:	"592056",
4561:	"592188",
6071:	"592579",
10496:	"592099",
2675:	"592846",
377:	"592609",
4987:	"584703",
8693:	"584347",
170:	"584568",
674:	"584568",
12405:	"584568",
5277:	"584363",
5278:	"584321",
6633:	"584801",
2264:	"584843",
2381:	"584967",
8598:	"584967",
6861:	"584592",
7435:	"584401",
7437:	"584711",
6292:	"584339",
8522:	"584355",
935:	"584746",
1586:	"584746",
6343:	"584746",
6341:	"584525",
6342:	"584789",
7661:	"584461",
7662:	"584461",
11403:	"584894",
11404:	"584894",
6231:	"586170",
6364:	"586170",
8786:	"586170",
11916:	"586170",
6065:	"586641",
9782:	"586641",
11168:	"586641",
6600:	"586099",
9401:	"586099",
9819:	"586561",
4830:	"586374",
7605:	"586498",
7626:	"586498",
11179:	"586498",
4319:	"586030",
10782:	"586030",
571:	"586790",
2106:	"586790",
5806:	"586790",
281:	"586421",
4826:	"586447",
5557:	"586595",
3611:	"586749",
4813:	"586749",
3781:	"586226",
5718:	"586188",
6968:	"586188",
4401:	"586480",
5089:	"586609",
1068:	"586048",
11177:	"586048",
2847:	"586323",
4829:	"586323",
5717:	"586323",
1361:	"586714",
4790:	"586714",
6818:	"586714",
10783:	"586714",
11167:	"586714",
2846:	"586331",
11668:	"586668",
12121:	"586668",
5394:	"586731",
4213:	"568864",
5542:	"599140",
678:	"598941",
9725:	"598941",
9866:	"598941",
2254:	"599077",
6905:	"599077",
10031:	"599077",
4240:	"598011",
7028:	"598445",
8262:	"598445",
8263:	"549665",
10094:	"549665",
5300:	"507423",
5301:	"507423",
11406:	"507181",
2977:	"598071",
9323:	"598071",
10890:	"598071",
9014:	"598356",
9239:	"598356",
293:	"598542",
1043:	"598542",
10891:	"598542",
1042:	"598747",
10889:	"598747",
6928:	"598674",
6929:	"598178",
6930:	"598178",
11423:	"598178",
10772:	"598101",
1232:	"598399",
11687:	"598399",
529:	"598551",
3045:	"598551",
9919:	"598551",
12533:	"598551",
1150:	"598135",
2549:	"598135",
6784:	"598691",
11688:	"598691",
1825:	"598321",
2856:	"598321",
3042:	"512184",
6305:	"598089",
6683:	"598089",
9494:	"598089",
9495:	"552518",
11579:	"552488",
1299:	"598160",
8775:	"598160",
9482:	"507237",
11214:	"507237",
11215:	"552623",
11573:	"552623",
10864:	"552640",
4680:	"552674",
12535:	"552674",
515:	"556971",
3183:	"598810",
9695:	"598810",
10868:	"598810",
1661:	"512028",
3845:	"512028",
10034:	"512028",
9022:	"554014",
10867:	"554014",
850:	"554928",
4178:	"554928",
6048:	"554928",
10095:	"554928",
10760:	"554928",
508:	"598062",
2412:	"598062",
3397:	"598062",
6785:	"598062",
8069:	"598062",
8452:	"598062",
9350:	"507091",
11212:	"507091",
3635:	"512176",
3846:	"512176",
5044:	"598453",
8899:	"598453",
9328:	"598453",
11200:	"598453",
9125:	"599905",
1099:	"599468",
2182:	"599867",
2402:	"568775",
7515:	"568546",
9717:	"568546",
11792:	"568546",
9865:	"599212",
11793:	"599212",
3244:	"599743",
7946:	"599743",
6181:	"568732",
11683:	"568732",
1595:	"599409",
10881:	"599409",
2047:	"500259",
341:	"599603",
10293:	"599506",
7099:	"568406",
10077:	"599964",
2472:	"599999",
11697:	"510131",
1450:	"570036",
1449:	"506451",
11698:	"510131",
2974:	"510343",
8909:	"510343",
845:	"507377",
846:	"507920",
5783:	"507920",
11690:	"507920",
567:	"507261",
2186:	"507261",
4348:	"512907",
8426:	"568368",
3684:	"507113",
8941:	"508144",
8943:	"508144",
11689:	"568279",
8942:	"568180",
1653:	"517607",
2959:	"514152",
4767:	"514152",
5009:	"516694",
5010:	"516694",
11592:	"516694",
6895:	"517321",
6820:	"513211",
4075:	"523640",
2761:	"513199",
11583:	"513199",
5204:	"552844",
7950:	"516686",
1567:	"521531",
11587:	"521531",
7960:	"517909",
7961:	"512231",
8384:	"512231",
708:	"542768",
10417:	"542768",
10420:	"542768",
2062:	"542784",
7360:	"542679",
10419:	"542679",
10421:	"542679",
6243:	"544566",
6327:	"544566",
10144:	"544566",
1097:	"545163",
5773:	"545163",
3347:	"544671",
6220:	"544990",
8967:	"544990",
9325:	"544990",
105:	"542725",
486:	"542725",
3604:	"542725",
6797:	"542725",
8480:	"542725",
11598:	"542725",
5291:	"542644",
6798:	"542644",
652:	"544787",
2003:	"544787",
2319:	"544787",
2318:	"542750",
5229:	"542750",
8317:	"542750",
3464:	"542865",
6445:	"542865",
1424:	"541711",
7642:	"541711",
3878:	"544949",
11208:	"544949",
8968:	"544396",
10145:	"544396",
2330:	"544302",
11211:	"544302",
11682:	"544302",
2329:	"542831",
8080:	"542989",
10777:	"542989",
3206:	"543021",
5189:	"544418",
5321:	"544418",
8972:	"544418",
11616:	"544418",
11618:	"544418",
11617:	"541648",
3202:	"545236",
5252:	"545236",
5983:	"545236",
469:	"544922",
4841:	"541800",
4914:	"541800",
4344:	"544698",
8191:	"544698",
11206:	"544698",
5491:	"542687",
8590:	"542687",
11205:	"542687",
10544:	"545210",
5948:	"542814",
7644:	"542814",
1025:	"545058",
9436:	"538744",
8124:	"585505",
4978:	"585271",
9745:	"585912",
11388:	"585912",
226:	"549622",
7431:	"549622",
12383:	"585068",
827:	"585343",
11383:	"585955",
4346:	"585882",
9358:	"585882",
9712:	"585980",
9893:	"585530",
9962:	"585530",
9963:	"585831",
12660:	"585831",
8315:	"585149",
11387:	"585661",
9435:	"585092",
11620:	"589195",
5634:	"588695",
8538:	"588695",
327:	"588385",
4314:	"589225",
5257:	"588849",
11614:	"588393",
12106:	"588393",
12373:	"588822",
11635:	"588598",
11636:	"588920",
3393:	"552402",
3895:	"500496",
305:	"500526",
4268:	"501646",
4859:	"501646",
6744:	"501646",
8246:	"501646",
10412:	"501646",
5872:	"500623",
10140:	"500623",
10770:	"502839",
10771:	"568961",
2925:	"503622",
10141:	"503622",
583:	"500861",
10142:	"500861",
11599:	"500861",
7396:	"504441",
7398:	"504441",
8247:	"547018",
7397:	"505013",
11218:	"505013",
3585:	"505111",
2022:	"501751",
10265:	"501751",
481:	"505269",
8275:	"505269",
4383:	"502235",
5367:	"502235",
7012:	"552411",
8503:	"552411",
9080:	"552411",
5368:	"505366",
7574:	"505366",
7534:	"505293",
11600:	"505293",
9015:	"505218",
9373:	"505218",
2653:	"552372",
4525:	"505587",
10413:	"505587",
7839:	"505501",
3888:	"504785",
9372:	"504785",
543:	"552160",
2558:	"502405",
3026:	"540544",
10897:	"540544",
575:	"540501",
7006:	"540501",
7546:	"540501",
7656:	"540501",
12668:	"540501",
2837:	"539961",
3025:	"539961",
4074:	"539961",
10896:	"539961",
6454:	"540226",
3460:	"540986",
1967:	"536521",
4313:	"536521",
1101:	"536288",
1339:	"540234",
12566:	"541478",
12606:	"541478",
3120:	"525979",
3121:	"532894",
3418:	"540510",
6747:	"540510",
5381:	"540773",
9008:	"536571",
10899:	"540871",
5261:	"540030",
10269:	"540030",
7949:	"540684",
9316:	"540684",
12568:	"540684",
11797:	"597589",
6887:	"597252",
5695:	"597554",
5696:	"597988",
4356:	"597481",
11800:	"597481",
11805:	"597481",
6188:	"597872",
5445:	"597368",
1869:	"597546",
10569:	"597911",
10117:	"597449",
12543:	"597449",
5697:	"597210",
3124:	"597813",
5446:	"597287",
3524:	"590126",
3551:	"590126",
3566:	"590126",
11657:	"590126",
2489:	"589845",
11651:	"589845",
2487:	"590207",
2488:	"590215",
329:	"590185",
7657:	"590185",
9961:	"590185",
12658:	"590185",
256:	"589829",
2023:	"589829",
9960:	"589829",
931:	"589608",
1688:	"589608",
1745:	"589454",
11646:	"589764",
11649:	"590100",
699:	"589683",
1798:	"589942",
8388:	"589942",
8387:	"590070",
7568:	"589691",
8386:	"590088",
9600:	"589314",
10154:	"589314",
10153:	"589519",
10158:	"589934",
11643:	"589934",
2486:	"589594",
10157:	"589594",
3147:	"589560",
11662:	"589292",
1650:	"589675",
11663:	"589675",
9810:	"589462"
}

# Přiřazení do sloupce 'street'
for item, city_code in doplnit_city_code.items():
    filtered.loc[filtered['id'] == item, 'city_code'] = city_code

# Volitelná kontrola
filtered.loc[doplnit_city_code.keys(), ['id', 'city_code']]

,id,city_code
1223,1224,554782.0
2206,2207,555771.0
3997,3998,585068.0
2443,2444,545341.0
4407,4408,537756.0
...,...,...
3147,3148,539228.0
11662,11663,589675
1650,1651,592731.0
11663,11664,586412.0


In [792]:
chybi_city_code = filtered[filtered['city_code'].isna()]
pocet = len(chybi_city_code)
chybi_city_code

,NazevZarizeni,DruhZarizeni,FormaPece,OborPece,Obec,Psc,Ulice,CisloDomovniOrientacni,Kraj,Lat,Lng,city_code,city_name,DruhZarizeni_clean,profese,id_typ_profese,nazev_profese,id
664,MUDr. KRYŠTOF POHL,Sam.ord.prakt.lékaře pro děti a dorost,ambulantní péče,pediatrie,Neratovice,nan,Dr. E. Beneše,1545,Středočeský kraj,NaN,NaN,NaN,NaN,Sam.ord.prakt.lékaře pro děti a dorost,pediatr,2,pediatr,665
935,Renturi s.r.o.,Samost. ordinace všeob. prakt. lékaře,ambulantní péče,všeobecné praktické lékařství,Kojetín,nan,Padlých hrdinů,NaN,Olomoucký kraj,NaN,NaN,NaN,NaN,Samost. ordinace všeob. prakt. lékaře,praktik,1,praktik,936
2937,Komplexní terapie s.r.o.,Samost. ordinace všeob. prakt. lékaře,primární ambulantní péče,všeobecné praktické lékařství,Ledvice,nan,nan,NaN,Ústecký kraj,NaN,NaN,NaN,NaN,Samost. ordinace všeob. prakt. lékaře,praktik,1,praktik,2938
4002,"MUDr. Gabriela Hniličková, s.r.o.",Samost. ordinace všeob. prakt. lékaře,primární ambulantní péče,všeobecné praktické lékařství,Letovice,nan,Komenského,1177/26,Jihomoravský kraj,NaN,NaN,NaN,NaN,Samost. ordinace všeob. prakt. lékaře,praktik,1,praktik,4003
7386,MUDr. Zbyněk Brada,Samost. ordinace všeob. prakt. lékaře,primární ambulantní péče,všeobecné praktické lékařství,Jimlín,nan,nan,NaN,Ústecký kraj,NaN,NaN,NaN,NaN,Samost. ordinace všeob. prakt. lékaře,praktik,1,praktik,7387
8205,MUDr. Václav Hlinovský,Samostatná ordinace PL - stomatologa,ambulantní péče,zubní lékařství,Rokycany,nan,Palackého,171,Plzeňský kraj,NaN,NaN,NaN,NaN,Samostatná ordinace PL - stomatologa,zubař,3,zubař,8206
8977,MUDr. Vladislav Sikora,Samost. ordinace všeob. prakt. lékaře,primární ambulantní péče,všeobecné praktické lékařství,Dolní Lutyně,nan,U Školy,432,Moravskoslezský kraj,NaN,NaN,NaN,NaN,Samost. ordinace všeob. prakt. lékaře,praktik,1,praktik,8978
11657,MUDr. Jana Dadáková,Samostatná ordinace PL - stomatologa,ambulantní péče,zubní lékařství,Drahany,79861,nan,206,Olomoucký kraj,49.433236,16.899387,NaN,NaN,Samostatná ordinace PL - stomatologa,zubař,3,zubař,11658


In [793]:
# Slovník: id → city_code
doplnit_city_code = {
    665: "535087",
    936: "568881",
    2938: "567655",
    4003: "581917",
    7387: "546011",
    8206: "559717",
    8978: "598968",
    11658: "589462"
}

# Přiřazení do sloupce 'street'
for item, city_code in doplnit_city_code.items():
    filtered.loc[filtered['id'] == item, 'city_code'] = city_code

# Volitelná kontrola
filtered.loc[doplnit_city_code.keys(), ['id', 'city_code']]

,id,city_code
665,666,569810.0
936,937,540315
2938,2939,554821.0
4003,4004,581879.0
7387,7388,529303.0
8206,8207,560383.0
8978,8979,569810.0
11658,11659,589250.0


In [794]:
filtered['city_code'] = filtered['city_code'].astype(str).str.replace(r'\.0$', '', regex=True)
filtered['city_code'].head()

0    554782
1    557153
2    561380
3    554804
4    567027
Name: city_code, dtype: object

In [795]:
import csv

# Sloupce, kde chceme chránit obsah (např. s adresami a městy)
sloupce_textove = ['Ulice', 'Obec', 'Psc', 'DruhZarizeni', 'FormaPece', 'OborPece', 'NazevZarizeni']

for sloupec in sloupce_textove:
    if sloupec in filtered.columns:
        filtered[sloupec] = filtered[sloupec].astype(str)

# Uložit CSV s uvozovkami
filtered.to_csv(
    r'C:\Users\Sabina\Sabi dokumenty\01 Czechitas\01 PYTHON\Python\Detektivky_vizualni_smog\sabi\data\doctors_pomocny.csv',
    index=False,
    encoding='utf-8-sig',   # BOM pro lepší kompatibilitu s Excel / Sheets
    quoting=csv.QUOTE_ALL,  # obalit všechny texty do uvozovek
    quotechar='"',
    sep=','
)

In [796]:
df = filtered.rename(columns={
    'NazevZarizeni': 'nazev',
    'city_code': 'kod_obec'
  })

df = df[['id', 'nazev', 'kod_obec', 'id_typ_profese']]
df.head(3)

,id,nazev,kod_obec,id_typ_profese
0,1,MDDr. Natalie Malinová,554782,3
1,2,MUDr. Stanislav Masný,557153,1
2,3,MEDILIFE s.r.o.,561380,1


In [797]:
print(df.dtypes)

id                 int64
nazev             object
kod_obec          object
id_typ_profese     int64
dtype: object


In [798]:
df['id'] = df['id'].astype(int)
df['nazev'] = df['nazev'].astype(str)
df['kod_obec'] = df['kod_obec'].astype(str)
df['id_typ_profese'] = df['id_typ_profese'].astype(int)


In [799]:
df.to_csv(
    r'C:\Users\Sabina\Sabi dokumenty\01 Czechitas\01 PYTHON\Python\Detektivky_vizualni_smog\sabi\data\ss_doctors_final_dataset.csv', 
    sep=";",                  # 1. oddělovač středník
    index=False,              # bez indexu
    quoting=1,                # 3. obalit všechny stringy do uvozovek
    quotechar='"',            # uvozovky
    encoding="utf-8"          # doporučené kódování
    )